In [18]:
import json
import pandas as pd
import requests
from datetime import date
import boto3
import time
import geopandas as gpd
import folium

In [23]:
aws_region = "ap-southeast-2"
schema_name = "fire_data"
table_name ='rfs_fire_data'
result_output_location = "s3://camgoo2-rfs-visualisation/query_results/"
bucket='camgoo2-rfs-visualisation'
athena_client = boto3.client('athena',region_name=aws_region)
s3_client = boto3.client('s3')

#convert sapefile to Geojson file.this does not need to be run anymore as is always done.

myshpfile = gpd.read_file('1259030001_lga11aaust_shape/LGA11aAust.shp')
myshpfile.to_file('LGA11aAust.geojson', driver='GeoJSON')

In [19]:
lga_boundaries = gpd.read_file('LGA11aAust.geojson')

In [24]:
def query_distinct_data(athena_client,result_output_location):
    query = "SELECT DISTINCT * from fire_data.rfs_fire_data where state in ('NSW','VIC','QLD')"
    response = athena_client.start_query_execution(
        QueryString=query,
        ResultConfiguration={"OutputLocation": result_output_location})
    return response["QueryExecutionId"]

In [26]:
execution_id = query_distinct_data(athena_client,result_output_location)

In [27]:
def distinct_data_df(pd,s3_client,bucket, response_path):
    #bucket = "camgoo2-rfs-visualisation"
    response_path = "query_results/"+execution_id+'.csv'

     
    # 's3' is a key word. create connection to S3 using default config and all buckets within S3
    obj = s3_client.get_object(Bucket= bucket, Key= response_path) 
    # get object and file (key) from bucket
    df = pd.read_csv(obj['Body']) # 'Body' is a key word
    df= df.dropna()
    return df

In [28]:
response_path = "query_results/"+execution_id+'.csv'

In [29]:
df = distinct_data_df(pd,s3_client,bucket, response_path)

In [43]:
#now trying to bring up the population when hovering over the LGA
bins = list(lga_boundaries["Tot_P_P"].quantile([0, 0.10, 0.3, 0.90, 1]))

fire_location = df[["latitude", "longitude"]]
map = folium.Map(location=[fire_location.latitude.mean(), fire_location.longitude.mean()],
                zoom_start=5, control_scale=True)
for index, location_info in fire_location.iterrows():
    folium.Marker([location_info["latitude"], location_info["longitude"]]
                      ,icon=folium.Icon(color="orange",icon = "fire")).add_to(map)
    #have used https://mapshaper.org/ to reduce the file size of polygons. I am now using 
    #reduced file LGA11aAust.json rather than LGA11aAust.geojson 

    
chloropeth = folium.Choropleth(
        geo_data='LGA11aAust.geojson',
        name='chloropeth',
        data=lga_boundaries,
        columns=['LGA_CODE11','Tot_P_P'], 
        key_on='feature.properties.LGA_CODE11', #Here we grab the geometries/county boundaries from the geojson file using the key 'coty_code' which is the same as county fips
        #threshold_scale=custom_scale, #use the custom scale we created for legend
        fill_color='YlOrRd',
        nan_fill_color="White", #Use white color if there is no data available for the county
        fill_opacity=0.7,
        line_opacity=0.2,
        #legend_name='New Cases Past 7 Days (Per 100K Population) ', #title of the legend
        highlight=True,
        #line_color='black'
        bins = bins,
).add_to(map)

chloropeth.geojson.add_child(folium.features.GeoJsonTooltip(['Tot_P_P']))
        
map

NameError: name 'choropleth' is not defined